# Inference

## Dependencies and imports

In [ ]:
import os

import numpy as np
import pandas as pd
from tqdm import tqdm

from openai import OpenAI

## Utility

In [ ]:
PROMPT = "Sei un dipendente pubblico che deve scrivere dei documenti istituzionali italiani per renderli semplici e comprensibili per i cittadini. Ti verrà fornito un documento pubblico e il tuo compito sarà quello di riscriverlo applicando regole di semplificazione senza però modificare il significato del documento originale. Ad esempio potresti rendere le frasi più brevi, eliminare le perifrasi, esplicitare sempre il soggetto, utilizzare parole più semplicii, trasformare i verbi passivi in verbi di forma attiva, spostare le frasi parentetiche alla fine del periodo."

In [ ]:
openai_client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

def build_prompt(_text_to_simplify):
    messages = [
        {"role": "system", "content": PROMPT},
        {"role": "user", "content": _text_to_simplify},
    ]
    return messages


def predict(_texts_to_simplify):
    prompts = [build_prompt(_text) for _text in _texts_to_simplify]

    outputs = []
    for prompt in tqdm(prompts):
        response = openai_client.chat.completions.create(
            model='gpt-3.5-turbo-0125',
            messages=prompt,
            stream=False,
            temperature=0.2,
            top_p=0.1
        )
        outputs.append(response.choices[0].message.content)
    return outputs

## Load datasets

In [ ]:
df = pd.read_csv('./texts/original.csv', encoding='utf-8')

## Random predictions

In [ ]:
for s in df.sample(1).to_dict(orient='records'):
  output = predict([s['original_text']])[0]
  print("\noriginal: ", s['original_text'])
  print("\nmodel: ", output)
  print('----------------')

100%|██████████| 1/1 [00:01<00:00,  1.87s/it]


original:  L'organizzazione dei servizi è definita dal progetto approvato dal Comune di Viterbo, disponibile e consultabile da tutti presso gli uffici del Comune di Viterbo, del soggetto gestore ATI e dell'azienda esecutrice dei servizi.

model:  Il progetto che definisce l'organizzazione dei servizi è stato approvato dal Comune di Viterbo. È possibile consultarlo presso gli uffici del Comune, del soggetto gestore ATI e dell'azienda che esegue i servizi.
----------------


## Run all predictions

In [ ]:
df['simplified_text'] = predict(df['original_text'].tolist())

100%|██████████| 619/619 [23:35<00:00,  2.29s/it]


## Save simplified datasets

In [ ]:
df.to_csv('./texts/gpt3_5.csv', index=False)